## 房屋資料抓取

In [ ]:
import requests
headers = {
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'   
}
res = requests.get('https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow=30&totalRows=2404&timestamp=1539754751297', headers = headers)

In [ ]:
res

In [ ]:
import pandas
pandas.DataFrame(res.json()['data']['house_list']).head(3)

In [ ]:
import time
houseurl = 'https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow={}&totalRows=2404&timestamp=1539754751297'

houseary = []
for i in range(0,int(2404 / 30)+ 1 ):
    res = requests.get(houseurl.format(i*30), headers = headers)
    df = pandas.DataFrame(res.json()['data']['house_list'])
    houseary.append(df)
    time.sleep(1)
    print(i)

In [ ]:
housedf = pandas.concat(houseary)

In [ ]:
housedf.count()

In [ ]:
housedf.to_excel('house591.xlsx')

## 房屋資料清理

In [ ]:
import pandas
housedf  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/chinalife/master/data/house591.xlsx')

In [ ]:
housedf.iloc[2]

In [ ]:
housedf = housedf[['area', 'room','floor','price','houseage']]

In [ ]:
housedf[['bedroom', 'livingroom', 'bathroom']] = housedf['room'].str.extract('(\d+)房(\d+)廳(\d+)衛')

In [ ]:
housedf[['current_floor', 'total_floor']] = housedf['floor'].str.extract('(\d+)F/(\d+)F')

In [ ]:
del housedf['room']
del housedf['floor']

In [ ]:
housedf.dropna(inplace=True)

In [ ]:
housedf.head()

In [ ]:
housedf.columns

In [ ]:
Y = housedf['price'].values
X = housedf[['area', 'bedroom', 'livingroom', 'bathroom', 'current_floor',
       'total_floor', 'houseage']]

In [ ]:
X['total_floor'] = X['total_floor'].astype(float)
X['bedroom'] = X['bedroom'].astype(float)
X['livingroom'] = X['livingroom'].astype(float)
X['bathroom'] = X['bathroom'].astype(float)
X['current_floor'] = X['current_floor'].astype(float)

## 資料分析

In [ ]:
import statsmodels.api as sm
X2  = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
predictorcols = X.columns

In [ ]:
import itertools
AICs = {}
for k in range(1,len(predictorcols)+1):
    for variables in itertools.combinations(predictorcols, k):
        predictors = X[list(variables)]
        predictors2 = sm.add_constant(predictors)
        est = sm.OLS(Y, predictors2)
        res = est.fit()
        AICs[variables] = res.aic

In [ ]:
from collections import Counter
c = Counter(AICs)
c.most_common()[::-1][0:3]

In [ ]:
import itertools
for variables in itertools.combinations(predictorcols, 4):
    print(variables)

## 連線資料庫

In [ ]:
import sqlite3 as lite
con = lite.connect('test.sqlite')
cur = con.cursor()
cur.execute('SELECT SQLITE_VERSION()')
data = cur.fetchone()
con.close()

In [ ]:
data

In [ ]:
import sqlite3 as lite
with lite.connect('test.sqlite') as con:
    cur = con.cursor()
    cur.execute('SELECT SQLITE_VERSION()')
    data = cur.fetchone()
#con.close()

In [ ]:
data

In [ ]:
import sqlite3 as lite 
with lite.connect("test.sqlite") as con: 
    cur = con.cursor() 
    cur.execute("DROP TABLE IF EXISTS PhoneAddress") 
    cur.execute("""CREATE TABLE PhoneAddress(
                    phone CHAR(10) PRIMARY KEY, 
                    address TEXT, 
                    name TEXT unique, 
                    age INT NOT NULL)""") 
    cur.execute("INSERT INTO PhoneAddress VALUES('0912173381','United State','Jhon Doe',53)") 
    cur.execute("INSERT INTO PhoneAddress VALUES('0928375018','Tokyo Japan','MuMu Cat',6)") 
    cur.execute("INSERT INTO PhoneAddress VALUES('0957209108','Taipei','Richard',29)") 


In [ ]:
import sqlite3 as lite 
with lite.connect("test.sqlite") as con: 
    cur = con.cursor() 
    cur.execute("SELECT phone, address from PhoneAddress")
    data = cur.fetchall()

In [ ]:
data

## Python and SQLServer


In [ ]:
import sqlite3 as lite 
with lite.connect("test.sqlite") as con: 
    cur = con.cursor() 
    cur.execute("SELECT phone, address from PhoneAddress")
    data1 = cur.fetchone()
    data2 = cur.fetchall()

In [ ]:
data1

In [ ]:
data2

In [ ]:
import sqlite3 as lite
import pandas
employee = [{'name':'Mary', 'age':23 , 'gender': 'F'},{'name':'John', 'age':33 , 'gender': 'M'}]
df = pandas.DataFrame(employee)
df

In [ ]:
df.info()

In [ ]:
with lite.connect('test.sqlite') as db:
    df.to_sql(name='employee', index=False, con=db, if_exists='replace')


## 儲存實例

In [ ]:
import pandas
housedf  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/chinalife/master/data/house591.xlsx')

In [ ]:
housedf = housedf[['area', 'room','floor','price','houseage']]

In [ ]:
housedf.info()

In [ ]:
with lite.connect('house.sqlite') as con:
    housedf.to_sql('house591', con = con)

In [ ]:
with lite.connect('house.sqlite') as con:
    df = pandas.read_sql_query('select * from house591 limit 3', con = con)

In [ ]:
df

In [ ]:
with lite.connect('house.sqlite') as con:
    df = pandas.read_sql_query('select avg(price) from house591', con = con)

In [ ]:
df

In [ ]:
with lite.connect('house.sqlite') as con:
    df = pandas.read_sql_query('select avg(price/area) from house591', con = con)

In [ ]:
df

## SQLAlchemy and SQLServer 
- https://docs.sqlalchemy.org/en/latest/core/engines.html

## 三大法人日報
- http://www.twse.com.tw/zh/page/trading/fund/T86.html

In [ ]:
import requests
res = requests.get('http://www.twse.com.tw/fund/T86?response=json&date=20181016&selectType=ALL&_=1539761083842')

In [ ]:
import pandas
df = pandas.DataFrame(res.json()['data'])

In [ ]:
df.columns = res.json()['fields']

In [ ]:
with lite.connect('test.sqlite') as con:
    df.to_sql('bigthree', con = con)

## 抓取公開資訊觀測站歷年變更登記

In [ ]:
import requests
from bs4 import BeautifulSoup
payload = {
'encodeURIComponent': '1',
'step': '1',
'firstin': '1',
'off': '1', 
'queryName': 'co_id',
'inpuType': 'co_id',
'TYPEK': 'all',
'co_id': '2823'    
}
res  = requests.post('http://mops.twse.com.tw/mops/web/ajax_t05st05', data = payload)
soup = BeautifulSoup(res.text, 'lxml')

In [ ]:
payload2 = {}
for item in soup.select_one('form').select('input[type=hidden]'):
    if item.get('value'):
        payload2[item.get('name')] = item.get('value')
payload2

In [ ]:
payload2['year'] = '1994'
payload2['month'] = '10'

In [ ]:
res = requests.post('http://mops.twse.com.tw/mops/web/ajax_t05st05', data = payload2)
res

In [ ]:
dfs = pandas.read_html(res.text)
dfs[1]

## 取得盤後交易資訊

## Rate Limiting
- https://www.nginx.com/blog/rate-limiting-nginx/

In [ ]:
import requests
import time
for i in range(1,10):
    res = requests.get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date=&type=&_=1539763053868')
    time.sleep(3)
    print(res)
#res.json()

## Proxy Rotate

In [ ]:
import requests
headers = {
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',  
 'origin': 'https://www.whatismyip.com',
 'referer': 'https://www.whatismyip.com/'   
}

proxies = {
  'http': 'http://196.17.26.204:8800',
  'https': 'http://196.17.26.204:8800',
}
res = requests.get('https://ipv4.whatismyip.com/', headers = headers, proxies = proxies)
res.text

In [ ]:
import requests
payload = {
'TYPEK': 'all',
'step': '0',
'stp': '1',
'firstin': 'true', 
'newstuff': '1'    
}
res = requests.post('http://mops.twse.com.tw/mops/web/t05sr01_1', data = payload)

In [ ]:
soup = BeautifulSoup(res.text, 'lxml')
payload2  = {}
for item in soup.select_one('form[name="fm_t05sr01_1"]').select('input[type=hidden]'):
    if item.get('value'):
        payload2[item.get('name')] = item.get('value')

In [ ]:
from bs4 import BeautifulSoup
def getDetail(payload):
    res2 = requests.post('http://mops.twse.com.tw/mops/web/ajax_t05sr01_1', data = payload)
    soup2 = BeautifulSoup(res2.text, 'lxml')
    #print(res2.text)
    return soup2.select_one('pre').text

In [ ]:
for rec in soup.select('input[value="詳細資料"]'):
    newpayload = payload2
    for item in rec.get('onclick').split(';'):
        if 'document' in item:
            if 'name' not in item.lower():
                key, value = item.replace('document.fm_t05sr01_1.', '').replace('.value', '').split('=')
                newpayload[key] = value.replace("'",'')
    #print(newpayload)
    news = getDetail(newpayload)
    print(news)
    break
    #print(news)

## 工作排程
- Windows: 工作排程器
- Linux/Mac: crontab
- http://linux.vbird.org/linux_basic/0430cron.php

## PTT 爬蟲

In [ ]:
import requests
cookies = {
    'over18': '1' 
}
res = requests.get('https://www.ptt.cc/bbs/Gossiping/M.1477049256.A.CEF.html', cookies = cookies)

In [ ]:
#res.text

## 財報狗一

In [ ]:
import requests
headers = {
  'Cookie': 'buuid=aEo3V1FLd3VQVytFRm5zUFJRa1J6YmIyRExsNHIzK2lMZURZWlB1bFQrQ1Zsa29oQTBDL3JyT3VsaEZZRExCWS0taDc3bFRGSUZDQjE4L1hKYnZIbVp3QT09--5c11ea3ba1ae9537ca134d9ce9dd0a93c94cae2f; easy_ab=04b5476a-f31e-415a-85f6-2444978be5a8; _ga=GA1.2.1476840361.1539768313; _gid=GA1.2.1539189090.1539768313; upgrade_browser=1; intercom-id-d5wdetwg=443922d8-11ba-4e97-9be1-0ad7225b9132; remember_user_token=W1szMTI2ODVdLCIkMmEkMTAkNWJqZGpkaVdubDRVY3JDdC5BQVJLdSIsIjE1Mzk3Njg4MDYuMTczNDIzMyJd--622b210368f395bfc55b38b965e75359babc5702; mp_distinct_id=312685; intercom-session-d5wdetwg=eFRMdmZKK2FnZ0pXRzlnRWVmQUgwQ3pSTGZaQmJwa1VlRWRvV2pXM2tpYWh1NVlsamUxMnlhTVhweHZ1ZGJQVS0tMDl5TVdBdkRYZUZ5YzBzRVI3RHRjZz09--f8d0aed123249329c99c02ebff9c204b2b381131; _statementdog_session_v2=YVBaQU1VSG9EK0R6UG9XNlRVVVU1eDhBSWtDY3l1OVJiaFdKZ3hsREs3VzRQT2hIaWZiK0txdUlvYkdZdmovUmFkd2dxK0daZDJFeXNPcTVGWFY1L21Cb3hlTFlzcklKVFQwcE9lcTFFa25HV0JmZzlDK1h2cmZOM09ZMGVKdk9kNzV3QVlCYUlnQ3FKdU5hbTFmYjduV2FhYXZEQTNlcCtFUzJsRFJIY0NoUURoNHo4ZGMxOTJ6TVlNcXhHN1pzZkM2aWd0OGlwMStVNXhIM2dqdk5uS24xb00ybTdKSmEvZXovMzFFWW12eWpyU05XRjlETmc5Z2Y2c2t6Zk1NZS0tUXVnZWdtZDlVcHRKZGNJRmtzUFN0UT09--1a7c1fd5b90f61d0d7b8693eddc548755ee4d188; mp_2fc60276bfe20ef798d15c46e498d765_mixpanel=%7B%22distinct_id%22%3A%20%22312685%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com.tw%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com.tw%22%7D; amplitude_id_0ab77a441fbc226251ef1a272380fcd7statementdog.com=eyJkZXZpY2VJZCI6ImYyYzliZGM5LWNhMDAtNGE5MC04NTk2LThhNjRiZGY2ZDViNVIiLCJ1c2VySWQiOiIzMTI2ODUiLCJvcHRPdXQiOmZhbHNlLCJzZXNzaW9uSWQiOjE1Mzk3NjgzMTI1OTgsImxhc3RFdmVudFRpbWUiOjE1Mzk3Njg5MjQ4NTYsImV2ZW50SWQiOjEyLCJpZGVudGlmeUlkIjoxMywic2VxdWVuY2VOdW1iZXIiOjI1fQ=='  
}
res = requests.get('https://statementdog.com/users/account', headers = headers)
#res.text

## 財報狗二

In [ ]:
import requests
from bs4 import BeautifulSoup
rs  = requests.session()
res = rs.get('https://statementdog.com/users/sign_in')
soup = BeautifulSoup(res.text, 'lxml')
payload = {}
for ele in soup.select('input[type=hidden]'):
    if ele.get('value'):
        payload[ele.get('name')] = ele.get('value')

In [ ]:
payload

In [ ]:
payload['user[email]'] = 'doma@oranek.com'
payload['user[password]'] = '1qaz2wsx'

In [ ]:
res2 = rs.post('https://statementdog.com/users/sign_in', data = payload)


In [ ]:
res3 = rs.get('https://statementdog.com/users/account')


In [ ]:
#res3.text

## Selenium
- ! pip install selenium
- https://chromedriver.storage.googleapis.com/2.43/chromedriver_win32.zip

In [ ]:
! pip install selenium

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('/opt/local/common/chromedriver')
driver.get('https://www.google.com')
driver.find_element_by_css_selector('input[name=q]').send_keys('中國人壽')
driver.find_element_by_css_selector('input[name=q]').submit()

In [ ]:
driver.close()

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('/opt/local/common/chromedriver')
driver.get('https://statementdog.com/users/sign_in')


In [ ]:
driver.find_element_by_css_selector('#user_email').send_keys('doma@oranek.com')

In [ ]:
driver.find_element_by_css_selector('#user_password').send_keys('1qaz2wsx')

In [ ]:
driver.find_element_by_css_selector('button.submit-btn').click()

In [ ]:
payload = {}
for rec in driver.get_cookies():
    payload[rec.get('name')] = rec.get('value') 

In [ ]:
headers = {
'Cookie':';'.join(['{}={}'.format(key, payload.get(key)) for key in payload])
}

In [ ]:
res = requests.get('https://statementdog.com/users/account', headers = headers)
#res.text

In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('/opt/local/common/chromedriver')
driver.get('https://irs.thsrc.com.tw/IMINT/?locale=tw')



In [ ]:
driver.find_element_by_css_selector('select[name=selectStartStation]').send_keys('台北')

In [ ]:
driver.find_element_by_css_selector('select[name=selectDestinationStation]').send_keys('嘉義')

In [ ]:
driver.find_element_by_css_selector('select[name=toTimeTable]').send_keys('19:00')

In [ ]:
driver.find_element_by_link_text('語音播放').click()

In [ ]:
import speech_recognition
r = speech_recognition.Recognizer()

voice = speech_recognition.AudioFile('/Users/davidchiu/Downloads/PronounceRandonNumber.wav')
with voice as source:
    audio = r.record(source)

captcha = r.recognize_google(audio, language='zh_TW')

In [ ]:
captcha

In [ ]:
import requests
from bs4 import BeautifulSoup
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'
}
rs = requests.session()
res = rs.get('http://bsr.twse.com.tw/bshtm/bsMenu.aspx', headers = headers)
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'lxml')

In [ ]:
payload = {}
for item in soup.select('input[type=hidden]'):
    if item.get('value'):
        payload[item.get('name')] = item.get('value')

In [ ]:
payload

In [ ]:
soup.select_one('#Panel_bshtm img')

In [ ]:
domain = 'http://bsr.twse.com.tw/bshtm/'
res2 = rs.get(domain + soup.select_one('#Panel_bshtm img')['src'])
with open('captcha.jpg', 'wb') as f:
    f.write(res2.content)

In [ ]:
from PIL import Image
Image.open('captcha.jpg')

In [ ]:
payload['TextBox_Stkno'] = 2330
#payload['CaptchaControl1'] = '6KJRD'
payload['__EVENTTARGET'] =''
payload['__EVENTARGUMENT'] =''
payload['__LASTFOCUS'] =''
payload['RadioButton_Normal'] ='RadioButton_Normal'
payload['btnOK'] = '查詢'


In [ ]:
res3 = rs.post('http://bsr.twse.com.tw/bshtm/bsMenu.aspx', data = payload, headers = headers)

In [ ]:
res4 = rs.get('http://bsr.twse.com.tw/bshtm/bsContent.aspx', headers = headers)
#print(res4.text)